In [34]:
import json
import logging
import os
from typing import List

import numpy as np
import pandas as pd
import pandas_gbq
import plotly.graph_objects as go
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 


ROOT_DIR = os.getcwd()
ROOT_DIR = os.path.dirname(ROOT_DIR)
#ROOT_DIR = os.path.realpath(os.path.join(os.path.dirname(__file__), ".."))
#LOG_PATH = os.path.join(ROOT_DIR, "logs/forecasting.log")
#logging.basicConfig(filename=LOG_PATH, encoding="utf-8", level=logging.DEBUG)


def _get_query_string(query_name: str):
    """
    Get a query from the queries folder, as a string
    :param query_name: The name of the query to use, without the .sql extension
    :return: The query as a string
    """
    with open(os.path.join(ROOT_DIR, f"queries/{query_name}.sql"), "r") as f:
        return f.read()


def _substitute_asin(query: str, asins: List[str] = None):
    """
    Create a where in clause from a list of ASINs
    :param query: The query to use as a string
    :param asins: The brands to filter by, as a list of strings
    :return: The query string with the where in clause
    """
    where = f" and asin in unnest({asins})" if asins else ""
    return query.replace("ASIN_SELECTION", where)


def get_calendar():
    calendar = _get_query_string("calendar")
    calendar_df, _ = _get_data_from_bq(calendar)
    return calendar_df


def _get_data_from_bq(query: str,):
    """
    Get data from BigQuery, using a query string
    :param query: The query to use
    :return: A pandas DataFrame
    """
    return pandas_gbq.read_gbq(query, project_id="yaba-data"), query

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
dataset = _get_data_from_bq(_get_query_string("everything_asin"))
print(dataset)

/usr/local/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2230: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


(     seller_region country_code        brand        asin reporting_date  \
0               na           us   matcha-dna  B077572GG8     2023-07-27   
1               na           us   matcha-dna  B077572GG8     2023-11-14   
2               na           us   matcha-dna  B077572GG8     2023-09-25   
3               na           us   matcha-dna  B077572GG8     2023-10-20   
4               na           us   matcha-dna  B077572GG8     2023-06-25   
...            ...          ...          ...         ...            ...   
1876            na           us  superscandi  B081NPK5Y2     2023-08-24   
1877            na           us  superscandi  B081NPK5Y2     2023-09-08   
1878            na           us  superscandi  B081NPK5Y2     2023-08-18   
1879            na           us  superscandi  B081NPK5Y2     2023-10-08   
1880            na           us  superscandi  B081NPK5Y2     2023-06-17   

      top_bsr_position  bot_bsr_position           sales  perdormance  units  \
0                 

In [28]:
data_top10 = pd.read_csv("../data/top10_data.csv")


In [19]:
data_top10.columns
data_top10.info

<bound method DataFrame.info of      seller_region country_code             brand        asin reporting_date  \
0               eu           uk  incite-nutrition  B00RXIIW7K     2023-10-02   
1               eu           uk  incite-nutrition  B00RXIIW7K     2023-11-15   
2               eu           uk  incite-nutrition  B00RXIIW7K     2023-12-15   
3               eu           uk  incite-nutrition  B00RXIIW7K     2023-10-22   
4               eu           uk  incite-nutrition  B00RXIIW7K     2023-11-13   
...            ...          ...               ...         ...            ...   
1876            na           us       igan-covers  B0817GWV8C     2023-09-17   
1877            na           us       igan-covers  B0817GWV8C     2023-11-14   
1878            na           us       igan-covers  B0817GWV8C     2023-06-06   
1879            na           us       igan-covers  B0817GWV8C     2023-10-24   
1880            na           us       igan-covers  B0817GWV8C     2023-10-12   

      t

In [20]:
data_top10.describe

<bound method NDFrame.describe of      seller_region country_code             brand        asin reporting_date  \
0               eu           uk  incite-nutrition  B00RXIIW7K     2023-10-02   
1               eu           uk  incite-nutrition  B00RXIIW7K     2023-11-15   
2               eu           uk  incite-nutrition  B00RXIIW7K     2023-12-15   
3               eu           uk  incite-nutrition  B00RXIIW7K     2023-10-22   
4               eu           uk  incite-nutrition  B00RXIIW7K     2023-11-13   
...            ...          ...               ...         ...            ...   
1876            na           us       igan-covers  B0817GWV8C     2023-09-17   
1877            na           us       igan-covers  B0817GWV8C     2023-11-14   
1878            na           us       igan-covers  B0817GWV8C     2023-06-06   
1879            na           us       igan-covers  B0817GWV8C     2023-10-24   
1880            na           us       igan-covers  B0817GWV8C     2023-10-12   

     

In [29]:
data_top10['reporting_date'] = pd.to_datetime(data_top10['reporting_date'])


In [30]:
data_top10['Month'] = data_top10['reporting_date'].dt.month
data_top10['Year'] = data_top10['reporting_date'].dt.year
data_top10['Date MonthYear'] = data_top10['reporting_date'].dt.strftime('%Y-%m')
#data_top10 = data_top10.drop(columns=['reporting_date'])

In [31]:
data_top10.head()

,seller_region,country_code,brand,asin,reporting_date,top_bsr_position,bot_bsr_position,sales,performance,units,...,pageviews_sde1,topbsr_units,botbsr_units,units_units,refunds_units,sessionsmobile_units,pageviews_units,Month,Year,Date MonthYear
0,eu,uk,incite-nutrition,B00RXIIW7K,2023-10-02,3.0,16.0,2817.04,1.000000,325.0,...,1.217804,0.009231,0.049231,1.0,-0.151692,1.135385,2.996923,10,2023,2023-10
1,eu,uk,incite-nutrition,B00RXIIW7K,2023-11-15,4.0,33.0,4436.46,1.150273,515.0,...,1.470796,0.007767,0.064078,1.0,-0.182155,1.273786,3.238835,11,2023,2023-11
2,eu,uk,incite-nutrition,B00RXIIW7K,2023-12-15,2.0,32.0,3444.11,0.898320,460.0,...,1.666155,0.004348,0.069565,1.0,-0.129152,1.054348,2.715217,12,2023,2023-12
3,eu,uk,incite-nutrition,B00RXIIW7K,2023-10-22,2.0,31.0,3973.09,1.034317,462.0,...,1.445392,0.004329,0.067100,1.0,-0.068658,1.283550,2.930736,10,2023,2023-10
4,eu,uk,incite-nutrition,B00RXIIW7K,2023-11-13,4.0,42.0,3417.13,0.916657,396.0,...,1.113257,0.010101,0.106061,1.0,-0.053965,1.136364,2.896465,11,2023,2023-11


In [32]:
df = data_top10
df.isnull().sum()

seller_region             0
country_code              0
brand                     0
asin                      0
reporting_date            0
top_bsr_position          0
bot_bsr_position          0
sales                    11
performance              11
units                    11
discounts                 0
cogs                     11
refunds                 394
sde1                      0
sessions_mobile          25
sessions_browser         25
sessions                 25
pageviews_mobile         25
pageviews_browser        25
pageviews                25
topbsr_sales             11
botbsr_sales             11
units_sales              11
refunds_sales           398
sessionsmobile_sales     26
pageviews_sales          26
topbsr_sde1               6
botbsr_sde1               6
units_sde1               11
refunds_sde1            394
sessionsmobile_sde1      25
pageviews_sde1           25
topbsr_units             11
botbsr_units             11
units_units              11
refunds_units       

In [33]:
# Display total values of all country 
pd.set_option('display.max_rows', None)
df['country_code'].value_counts()

country_code
us    1254
uk     627
Name: count, dtype: int64

In [ ]:
country_names = df.Country.value_counts().index
country_val = df.Country.value_counts().values
# Pie Chart for top 20 country
fig,ax = plt.subplots(figsize=(9,9))
ax.pie(country_val[:20],labels=country_names[:20],autopct='%1.2f%%')
plt.show()